In [5]:
import keras
from keras.applications.vgg16 import VGG16
import keras.models import Sequential
from keras.models import Dense, Activation
from keras.model import

1 dataset

In [6]:
base_model = VGG16(include_top=True, weights='imagenet', input_shape=(224, 224, 3))
base_model.summary()

553467904/553467096 [==============================] - 3s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [5]:
!git clone https://github.com/kaa/coins-dataset

Cloning into 'coins-dataset'...
remote: Enumerating objects: 3001, done.
remote: Total 3001 (delta 0), reused 0 (delta 0), pack-reused 3001
Receiving objects: 100% (3001/3001), 223.70 MiB | 42.89 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [6]:
import os

images_classes_path = '/content/coins-dataset/classified/train'

images_classes_list = os.listdir(images_classes_path)

num_classes = len(images_classes_list)
print(" we use {} classes".format(num_classes))


 we use 8 classes


In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import matplotlib.image as mpimg
import numpy as np

preprocess

In [8]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image      
import random

In [13]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range =20,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        validation_split = 0.4,
       )

train_generator = train_datagen.flow_from_directory(
        '/content/coins-dataset/classified/train',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')
validation_generator = train_datagen.flow_from_directory(
        '/content/coins-dataset/classified/train',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        '/content/coins-dataset/classified/test',
        target_size=(224, 224),
        shuffle=False,
        batch_size=1,
        class_mode='categorical')

filenames = test_generator.filenames
nb_samples = len(filenames)

Found 1291 images belonging to 8 classes.
Found 1291 images belonging to 8 classes.
Found 1539 images belonging to 8 classes.
